<a href="https://colab.research.google.com/github/ManJ-PC/redes-neurais/blob/master/Gerar_dataset_e_fine_tuning_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# bibliotecas importantes

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 openai


In [ ]:
import os
import openai
import random
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import pipeline
import pandas as pd
from google.colab import drive

drive.mount("/content/drive")
label_base = "grafos"
caminho_base = f"/content/drive/MyDrive/Llama_models/{label_base}"
caminho_dataset = "/content/drive/MyDrive/Datasets"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
openai_key = "COLOQUE SUA CHAVE DA API OPEN AI AQUI"
openai.api_key = openai_key
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_key,
)

# preparação e geração  dos dados

Vamos gerar um contexto sobre um assunto. Vamos pegar uma apostila explicando o que são grafos. Vamos fazer questões sobre isso tb.

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4",
)

In [ ]:
chat_completion.choices[0].message.content

'This is a test.'

In [ ]:
prompt = "Um modelo que aborda uma pergunta sobre grafos, em português, e responde com uma resposta bem fundamentada e passo a passo em português."
temperature = .4
number_of_examples = 16

In [ ]:
def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"Você está gerando dados que serão usados ​​para treinar um modelo de aprendizado de máquina."\
            "\n\nVocê receberá uma descrição de alto nível do modelo que deseja treinar e, a partir disso, gerará amostras de dados, cada uma com um prompt/par de respostas."\
            "\n\nVocê fará isso neste formato:\n```\nprompt\n-----------\n$prompt_goes_here\n---------- -\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\n"\
            "Apenas um par prompt/resposta deve ser gerado por turno.\n\n"\
            "Para cada turno, torne o exemplo um pouco mais complexo que o anterior, garantindo a diversidade.\n\n"\
            "Certifique-se de que suas amostras sejam únicas e diversificadas, mas de alta qualidade e complexas o suficiente para treinar um bom modelo de desempenho.\n\n"\
            f"Aqui está o tipo de modelo que queremos treinar:\n`{prompt}"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = client.chat.completions.create(
        messages=messages,
        model="gpt-4",
        temperature=temperature,
        max_tokens=1354
    )

    return response.choices[0].message.content

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Gerando exemplo {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

Gerando exemplo 0
Gerando exemplo 1
Gerando exemplo 2
Gerando exemplo 3
Gerando exemplo 4
Gerando exemplo 5
Gerando exemplo 6
Gerando exemplo 7
Gerando exemplo 8
Gerando exemplo 9
Gerando exemplo 10
Gerando exemplo 11
Gerando exemplo 12
Gerando exemplo 13
Gerando exemplo 14
Gerando exemplo 15
['prompt\n-----------\nO que é um grafo em ciência da computação?\n---------- -\nresponse\n-----------\nUm grafo, no contexto da ciência da computação, é uma estrutura de dados abstrata que é usada para representar conexões ou relacionamentos entre objetos. Um grafo é composto por vértices (também chamados de nós) e arestas (também chamadas de links ou arcos). Cada aresta conecta dois vértices, indicando um relacionamento entre eles. Os grafos podem ser direcionados (onde as arestas têm uma direção) ou não direcionados (onde as arestas não têm direção). Eles são usados \u200b\u200bem uma ampla variedade de aplicações, incluindo redes de computadores, bancos de dados, inteligência artificial e muit

In [ ]:
# Initialize lists to store prompts and responses
prompts = []
responses = []

for example in prev_examples:
  try:
    split_example = example.split('-----------')
    question = split_example[1].replace("\n---------- -\nresponse\n", "")
    answer = split_example[2]
    prompts.append(question.strip())
    responses.append(answer.strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head(10)

There are 16 successfully-generated examples. Here are the first few:


,prompt,response
0,O que é um grafo em ciência da computação?,"Um grafo, no contexto da ciência da computação..."
1,O que é um grafo bipartido e como podemos veri...,Um grafo bipartido é um tipo especial de grafo...
2,O que é um ciclo Hamiltoniano em um grafo e co...,Um ciclo Hamiltoniano em um grafo é um ciclo q...
3,O que é um grafo ponderado e como podemos repr...,Um grafo ponderado é um grafo em que cada ares...
4,Como funciona o algoritmo de Floyd-Warshall pa...,O algoritmo de Floyd-Warshall é um algoritmo q...
5,O que é um grafo acíclico direcionado (DAG) e ...,Um grafo acíclico direcionado (DAG) é um grafo...
6,Como funciona o algoritmo de busca em largura ...,"O algoritmo de busca em largura, também conhec..."
7,O que é um grafo euleriano e como podemos veri...,Um grafo euleriano é um grafo que possui um ci...
8,O que é um grafo conexo e como podemos verific...,Um grafo é dito conexo se existir um caminho e...
9,Como funciona o algoritmo de Kruskal para enco...,O algoritmo de Kruskal é um algoritmo que enco...


In [ ]:
# salvando dataset
df.to_csv(f"{caminho_dataset}/{label_base}.csv", sep=";", index=False)

In [ ]:
def generate_system_message(prompt):

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "Você receberá uma descrição de alto nível do modelo que estamos treinando e, a partir disso, gerará um prompt "\
                      "de sistema simples para esse modelo usar. Lembre-se, você não está gerando a mensagem do sistema "\
                      "para geração de dados – você está gerando a mensagem do sistema para usar para inferência. "\
                      "Um bom formato a seguir é `Dado $INPUT_DATA, você $WHAT_THE_MODEL_SHOULD_DO.`.\n\n"\
                      "Faça-o o mais conciso possível. Não inclua nada além do prompt do sistema em sua resposta.\n\n"\
                      "Por exemplo, nunca escreva: `\"$SYSTEM_PROMPT_HERE\"`.\n\nDeve ser algo como: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message.content

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Dada a pergunta sobre grafos em português, você deve fornecer uma resposta bem fundamentada e passo a passo em português.`. Feel free to re-run this cell if you want a better result.


# Separando treino e teste

In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json(f'{caminho_dataset}/{label_base}_train.jsonl', orient='records', lines=True)
test_df.to_json(f'{caminho_dataset}/{label_base}_test.jsonl', orient='records', lines=True)

# parametros llama


In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-chat-hf"
# The instruction dataset to use
dataset_name = f'{caminho_dataset}/{label_base}_train.jsonl'
# Fine-tuned model name
new_model = "llama-2-7b-custom"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
################################################################################
# TrainingArguments parameters
################################################################################
# Output directory where the model predictions and checkpoints will be stored
output_dir = f"{caminho_base}/results"
# Number of training epochs
num_train_epochs = 1
# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False
# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files=f'{caminho_dataset}/{label_base}_train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files=f'{caminho_dataset}/{label_base}_test.jsonl', split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

# Logando no hugging faces


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Computando modelo

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(f"{caminho_base}/{new_model}")
trainer.tokenizer.save_pretrained(f"{caminho_base}/{new_model}")



You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


In [ ]:

# testando modelo
logging.set_verbosity(logging.CRITICAL)
question = "o que é um grafo direcionado?"
prompt = f"[INST] <<SYS>>\n{system_message}. Responda em português. Com no máximo 300 tokens\n<</SYS>>\n\n{question}. [/INST]"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(prompt)
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] <<SYS>>
Dada a pergunta sobre grafos em português, você deve fornecer uma resposta bem fundamentada e passo a passo em português.. Responda em português. Com no máximo 300 tokens
<</SYS>>

o que é um grafo direcionado?. [/INST]  Ah, uma pergunta interessante sobre grafos! 🤔

Um grafo direcionado é um tipo de grafo em que os nodos são conectados por caminhos direcionados, ou seja, os caminhos que conectam dois nodos têm uma direção específica. Isso significa que, diferentemente de um grafo não direcionado, em que os caminhos podem ser percorridos em ambas as direções, em um grafo direcionado, os caminhos podem ser percorridos apenas em uma direção. 🚗

Por exemplo, consideremos um grafo com quatro nodos: A, B, C e D. Em um grafo não direcionado, podemos percorrer caminhos de A para B, B para A, C para D e D para C, sem preocupar-se em qual a direção é.
